In [1]:
PROJECT_NAME = 'mobilenet_v3_dropout_EXP'

# Import

In [2]:
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary


In [3]:
import timm

In [4]:
try:
    import wandb
except:
    !pip install wandb
    import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
import pandas as pd
import os
from pathlib import Path


import torch

from torch.utils.data import DataLoader

"""
Contains functions for training and testing a PyTorch model.
"""

from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [6]:
from dataset_helper import ImageFolderCustom
from model_engine import Engine
from helper_plot import plot_torch_hist

# Directory

In [7]:
data_path = Path('/kaggle/input/oral-dataset/patches')

In [8]:
image_dir = data_path / 'images'
train_df = pd.read_csv(data_path/'sabpatch_parsed_folders.csv')
train_ds = train_df[['path','lesion']]
test_df = pd.read_csv(data_path/'sabpatch_parsed_test.csv')
test_ds = test_df[['path','lesion']]

# Global Parameters

In [9]:
ALPHA = 0.001 ## Learning Rate
EPOCH = 50  ## Epochs
BATCH_SIZE = 32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DROPOUT = 0.25

MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

In [10]:
model = timm.create_model(
    'mobilenetv3_large_100.ra_in1k',
    pretrained=True,
    num_classes=3,  # remove classifier nn.Linear
    drop_rate = DROPOUT
)

data_config = timm.data.resolve_model_data_config(model)
train_transforms = timm.data.create_transform(**data_config, 
                                        is_training=True,
                                                                
)
test_transforms = timm.data.create_transform(**data_config, 
                                             is_training=False)

model.safetensors:   0%|          | 0.00/22.1M [00:00<?, ?B/s]

In [11]:
# import torchvision

In [12]:
# model = torchvision.models.mobilenetv3(pretrained=True)
# model

In [13]:
# feats_list = list(model.features)
# new_feats_list = []
# for feat in feats_list:
#     new_feats_list.append(feat)
#     if isinstance(feat, nn.Conv2d):
#         new_feats_list.append(nn.Dropout(p=0.5, inplace=True))

# # modify convolution layers
# model.features = nn.Sequential(*new_feats_list)


In [14]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), 
                                      ratio=(0.75, 1.3333), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=(0.6, 1.4), 
                contrast=(0.6, 1.4), 
                saturation=(0.6, 1.4), hue=0),
    transforms.ToTensor(),
    transforms.Normalize(mean=([0.4850, 0.4560, 0.4060]), 
              std=([0.2290, 0.2240, 0.2250])),
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=([0.4850, 0.4560, 0.4060]), 
              std=([0.2290, 0.2240, 0.2250])),
])

In [15]:
# from torch.utils.data import default_collate
# from torchvision.transforms import v2

# def collate_fn(batch):
#     return cutmix_or_mixup(*default_collate(batch))

# cutmix = v2.CutMix(num_classes=3)
# mixup = v2.MixUp(num_classes=3)
# cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])

In [16]:
    train_data = ImageFolderCustom(targ_dir = image_dir,
                                          path_df = train_ds,
                                          transform= train_transforms)

    test_data = ImageFolderCustom(targ_dir = image_dir,
                                          path_df = test_ds,
                                          transform= test_transforms)
    
    classes, class_to_idx = train_data.find_classes()

/kaggle/usr/lib/dataset_helper/dataset_helper.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.img_df['path'] = self.check_path(targ_dir)


In [17]:
train_dataloader = DataLoader(dataset=train_data, # use custom created train Dataset
                                        batch_size=BATCH_SIZE, # how many samples per batch?
                                        num_workers=0, # how many subprocesses to use for data loading? (higher = more)
                                        shuffle=True, # shuffle the data?
                                        )

test_dataloader = DataLoader(dataset=test_data, # use custom created test Dataset
                                        batch_size=BATCH_SIZE,
                                        num_workers=0,
                                        shuffle=False) # don't usually need to shuffle testing data


In [18]:
    wandb.init(
    project=PROJECT_NAME,
    config={
            "epochs": EPOCH,
            "batch_size": BATCH_SIZE,
            "lr": ALPHA,
            "architecture": "CNN",
            "dropout": DROPOUT
            })


wandb: Currently logged in as: jhashulabh. Use `wandb login --relogin` to force relogin


In [ ]:
loss_fn = nn.CrossEntropyLoss(reduction='sum') # computes the cross entropy loss between input logits and target.

optimizer = torch.optim.Adam(model.parameters(), lr = ALPHA)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Setup training and save the results
Engine = Engine(model=model, loss_fn=loss_fn, optimizer=optimizer, device=device, early_stopping=True , config = wandb.config)
result = Engine.train(train_dataloader=train_dataloader, test_dataloader=test_dataloader, epochs=EPOCH, verbose=True, patience=5)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

Epoch 1/50:   0%|          | 0/98 [00:00<?, ?batch/s]

Epoch 1/50:   0%|          | 0/20 [00:00<?, ?batch/s]

Epoch: 1 | train_loss: 84.9075 | train_acc: 0.5354 | test_loss: 48.4142 | test_acc: 0.6093


Epoch 2/50:   0%|          | 0/98 [00:00<?, ?batch/s]

Epoch 2/50:   0%|          | 0/20 [00:00<?, ?batch/s]

Epoch: 2 | train_loss: 45.9272 | train_acc: 0.5848 | test_loss: 37.1050 | test_acc: 0.6858


Epoch 3/50:   0%|          | 0/98 [00:00<?, ?batch/s]

Epoch 3/50:   0%|          | 0/20 [00:00<?, ?batch/s]

Epoch: 3 | train_loss: 43.0773 | train_acc: 0.6043 | test_loss: 33.0486 | test_acc: 0.6842


Epoch 4/50:   0%|          | 0/98 [00:00<?, ?batch/s]

In [ ]:
result_df = pd.DataFrame(result)
# result_df.head()

In [ ]:
plot_torch_hist(result_df)